In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix, identity, kron
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh
from scipy.sparse.linalg import expm

def build_operator(N, op_type, index):
    """Builds an operator (sx or sz) acting on a specific atom index."""
    sz = csr_matrix([[1, 0], [0, -1]])   #defining Pauli matrices
    sx = csr_matrix([[0, 1], [1, 0]])
    I2 = identity(2)
    
    target = sz if op_type == 'sz' else sx     #determines which Pauli amtrix to use on the code 
    op_list = [I2] * N          #creates N copies of the Hmailtonian 
    op_list[index] = target     #picks one of the atoms to act upon with the Pauli matrices
    
    res = op_list[0]
    for j in range(1, N):
        res = kron(res, op_list[j], format='csr') #performs a tensor product operation on all matrices, initialised with op_list[0]
    return res

def build_zz(N, i, j):   #defines the interaction terms between two atoms, 
    sz = csr_matrix([[1, 0], [0, -1]])
    I2 = identity(2)

    ops = [I2] * N
    ops[i] = sz
    ops[j] = sz

    res = ops[0]
    for k in range(1, N):
        res = kron(res, ops[k], format='csr')

    return res

def build_hamiltonian(N, J, g):
    dim = 2**N
    H = csr_matrix((dim, dim))
    # Field Term (X-direction)
    for i in range(N):
        H -= g * build_operator(N, 'sx', i)
    #Interaction Term (Z-direction)
    for i in range(N-1):
        H -= J * build_zz(N, i, i+1)
    # PINNING FIELD: Add a tiny bit of Z-field to the first atom 
    # This prevents the "stripey" random flipping between Red and Blue.
    H -= initialisation * build_operator(N, 'sz', 0)
    
    return H


N = 4       # Number of atoms
J = 1  # Coupling strength
beta = 0.01  #temperature parameter 
trotter_value = 1.5
initialisation = 0.005
g_values = np.linspace(0,1.5 ,60)
all_atom_data = []
m_plot_data = []
partition_functions = []
gaps_plot_data1 = []
gaps_plot_data2 = []
history = []

tau_total = 1 # Total "imaginary time" (higher is more accurate/colder)
steps = 100       # Number of Trotter slices
d_tau = tau_total / steps

for g in g_values:

    H = build_hamiltonian(N, J, g)
    if np.isclose(g, 0): 
        energy_val, _ = eigsh(H, k=1, which='SA')
        gs_energy = energy_val[0]
        print(f"Ground state energy at classical limit (g=0) = {gs_energy:.4f}")

    eigvals = eigh(H.toarray(), eigvals_only=True)
    gap1 = eigvals[2] - eigvals[0]
    gaps_plot_data1.append(gap1)
    gap2 = eigvals[1] - eigvals[0]
    gaps_plot_data2.append(gap2)
    # this code calculates the partition function for varying values of the transverse field, g
    # Complete full diagonalization to get all energy levels
    all_energies = eigh(H.toarray(), eigvals_only=True)

    # --- TROTTER METHOD START ---
    if np.isclose(g, trotter_value, atol = 0.01):
        
        # 1. Build the FULL Interaction Matrix (Sum of all ZZ terms)
        dim = 2**N
        H_int = csr_matrix((dim, dim))
        for i in range(N-1):
            H_int -= J * build_zz(N, i, i+1)
        
        # 2. Build the FULL Field Matrix (Sum of all X terms)
        H_field = csr_matrix((dim, dim))
        for i in range(N):
            H_field -= g * build_operator(N, 'sx', i)

        # 3. Create the Trotter Gates
        gate_int = expm(-d_tau * H_int.toarray())
        gate_field = expm(-d_tau * H_field.toarray())
    
        # Start with a random state
        psi_trotter = np.random.rand(2**N) + 1j*np.random.rand(2**N)
        psi_trotter /= np.linalg.norm(psi_trotter)
    
        # Apply the Trotter steps
        for _ in range(steps):
            psi_trotter = gate_int @ psi_trotter
            psi_trotter = gate_field @ psi_trotter
            psi_trotter /= np.linalg.norm(psi_trotter) 
            history.append(np.abs(psi_trotter)**2)
        
        # Calculate Energy
        e0_trotter = np.vdot(psi_trotter, H @ psi_trotter).real

        
        
        # Use E_min from your eigh results for the print statement
        print(f"g={g:.2f} | ED E0: {gs_energy:.4f} | Trotter E0: {e0_trotter:.4f}")

    ###### END TROTTER METHOD 
    
    # Calculate the actual lnZ
    E_min = np.min(all_energies)
    Z_shifted = np.sum(np.exp(-beta * (all_energies - E_min)))
    actual_lnZ = np.log(Z_shifted) - (beta * E_min)

    
    partition_functions.append(actual_lnZ) 

    # Get the ground state 
    energy, states = eigsh(H, k=1, which='SA')
    psi = states[:, 0]
    # psi = psi / np.linalg.norm(psi)
    
    # Calculate <Sz> for EACH of the 8 atoms
    current_g_step_spins = []
    for i in range(N):
        sz_op = build_operator(N, 'sz', i)
        # Expectation value: <psi|Sz|psi>
        expectation = np.vdot(psi, sz_op.dot(psi)).real
        current_g_step_spins.append(expectation)

    all_atom_data.append(current_g_step_spins)
            # 1. Average the spins of all N atoms for this specific g
    magnetization = np.mean(current_g_step_spins)
    
    # 2. Store it in a new list (you'll need to initialize 'm_plot_data = []' outside the loop)
    m_plot_data.append(magnetization)


plt.figure(figsize=(12, 6))

sns.heatmap(np.array(all_atom_data).T, 
            xticklabels=np.round(g_values, 2), 
            yticklabels=[f"Atom {i}" for i in range(N)],
            cmap='coolwarm', vmin=-1, vmax=1, center=0)

plt.title(f"Quantum Phase Transition: Individual Atom States (N={N})")
plt.xlabel("Transverse Field Strength (g)")
plt.ylabel("Atom Position in Chain")
plt.show()

plt.plot(g_values, m_plot_data)
plt.show()

dM_dg = np.abs(np.diff(m_plot_data) / np.diff(g_values))

# 2. Find the index of the maximum slope
# np.argmax gives us the index where the drop is steepest
max_slope_idx = np.argmax(dM_dg)

# 3. Extract the critical g value
# We use the midpoint between the two g-values where the diff was taken
g_critical = (g_values[max_slope_idx] + g_values[max_slope_idx + 1]) / 2

print(f"Estimated Critical Point (g_c): {g_critical:.4f}")

plt.plot(g_values, partition_functions)
plt.show()

plt.spy(H, markersize=1)
plt.title("Visualizing the Sparse Hamiltonian Matrix")
plt.show()

plt.plot(g_values, gaps_plot_data1, c = 'red')
plt.plot(g_values, gaps_plot_data2, c = 'blue')
plt.show()

plt.imshow(np.array(history).T, aspect='auto', cmap='viridis')
plt.colorbar(label='Probability')
plt.xlabel('Time Step')
plt.ylabel('State Index (0-15)')
plt.title('State Evolution in Imaginary Time')


Validation steps: 

The switch of magentization should occur at g=1 for an infinite chain, but becasue there is a high proportion of atoms on the edge of the chain, the energy per spin is a lot less. These atoms only have one bond, making the transfer point of the crossover lower than one. As the system size increases, the critical point gets closer and closer to occurring when g = 1, J = 1. 

We have validated that the ground state energy (in absence of the g field) equals -(N-1)J. 

Research into different values of beta and the effect they have on the Z value. Note at high temperatures the partition function is pretty constant, but as the temperature decreases the physics becomes more interesting... 

quantum phase transition at a critical point... 